In [11]:
#https://www.kaggle.com/errearanhas/topic-modelling-lda-on-elon-tweets/notebook
import os
import numpy as np 
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go

#init_notebook_mode(connected=True) #do not miss this line

from gensim import corpora, models, similarities

import warnings
warnings.filterwarnings("ignore")

In [1]:
#datafile = 'Bahrain_tweets_cleaned.csv'
datafile = 'Bahrain_tweets_cleaned_Arabic.csv'

In [3]:
import pandas as pd
tweets = pd.read_csv(datafile)
tweets.head(10)

,status_id,text,char_length
0,1056255227857395717,ختام فعاليه الفرقه الموسيقيه العسكريه لقوه دفا...,146
1,1056155633672744960,يرجي كتابه عنوان الموقع كاملا في متصفح الانترنت,47
2,1055985919273115653,تعرف علي مترشحي الانتخابات النيابيه والبلديه ع...,120
3,1055985156295745536,يرجي كتابه عنوان الموقع كاملا في متصفح الانترنت,47
4,1055773814158778368,مواعيد تلقي الطعون علي اعتراضات المترشحين بوزا...,179
5,1055033336719138816,كونوا علي الموعد يومي الخميس والجمعه القادمين ...,136
6,1055032537242914816,نلبي الواجب انتخابات البحرين الانتخابات النياب...,78
7,1054452449220681730,تسجل اكبر اقبال علي الترشح لانتخابات النيابيه ...,137
8,1054437940468871169,ونوه المدير التنفيذي لانتخابات المستشار نواف ح...,236
9,1054437937805512704,وحول عد المترشحين لمجالس البلديه حسب المحافظات...,133


In [4]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15093 entries, 0 to 15092
Data columns (total 3 columns):
status_id      15093 non-null int64
text           15093 non-null object
char_length    15093 non-null int64
dtypes: int64(2), object(1)
memory usage: 353.8+ KB


In [5]:
range(len(tweets['text']))

range(0, 15093)

***Tweet Activity Over Years***

In [6]:
corpus=[]
a=[]
for i in range(len(tweets['text'])):
        a=tweets['text'][i]
        corpus.append(a)

In [7]:
corpus[0:5]

['ختام فعاليه الفرقه الموسيقيه العسكريه لقوه دفاع البحرين التي اقيمت يومي الخميس والجمعه الماضين بمجمع السيف 🇧🇭 نلبي الواجب انتخابات البحرين البحرين',
 'يرجي كتابه عنوان الموقع كاملا في متصفح الانترنت',
 'تعرف علي مترشحي الانتخابات النيابيه والبلديه عبر الموقع الالكتروني الرسمي لانتخابات نلبي الواجب انتخابات البحرين البحرين',
 'يرجي كتابه عنوان الموقع كاملا في متصفح الانترنت',
 'مواعيد تلقي الطعون علي اعتراضات المترشحين بوزاره العدل مساء يوم الجمعه ظهرا يوم السبت نلبي الواجب الطعون الانتخابيه انتخابات البحرين الانتخابات النيابيه الانتخابات البلديه البحرين']

In [8]:
import gensim
import logging
import tempfile

TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

from gensim import corpora
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

C:\Users\Ayah\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Folder "C:\Users\Ayah\AppData\Local\Temp" will be used to save temporary dictionary and corpus.


In [9]:
from nltk.corpus import stopwords
from string import punctuation

# remove common words and tokenize
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]

In [12]:
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'bahrain.dict'))  # store the dictionary, for future reference
#print(dictionary)

2018-11-03 14:09:31,278 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-11-03 14:09:31,851 : INFO : adding document #10000 to Dictionary(22318 unique tokens: ['اقيمت', 'البحرين', 'التي', 'الخميس', 'السيف']...)
2018-11-03 14:09:32,134 : INFO : built Dictionary(27867 unique tokens: ['اقيمت', 'البحرين', 'التي', 'الخميس', 'السيف']...) from 15093 documents (total 281196 corpus positions)
2018-11-03 14:09:32,134 : INFO : saving Dictionary object under C:\Users\Ayah\AppData\Local\Temp\bahrain.dict, separately None
2018-11-03 14:09:32,173 : INFO : saved C:\Users\Ayah\AppData\Local\Temp\bahrain.dict


In [13]:
#print(dictionary.token2id)

In [14]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'elon.mm'), corpus)  # store to disk, for later use

2018-11-03 14:09:40,642 : INFO : storing corpus in Matrix Market format to C:\Users\Ayah\AppData\Local\Temp\elon.mm
2018-11-03 14:09:40,646 : INFO : saving sparse matrix to C:\Users\Ayah\AppData\Local\Temp\elon.mm
2018-11-03 14:09:40,647 : INFO : PROGRESS: saving document #0
2018-11-03 14:09:40,697 : INFO : PROGRESS: saving document #1000
2018-11-03 14:09:40,748 : INFO : PROGRESS: saving document #2000
2018-11-03 14:09:40,821 : INFO : PROGRESS: saving document #3000
2018-11-03 14:09:40,903 : INFO : PROGRESS: saving document #4000
2018-11-03 14:09:40,960 : INFO : PROGRESS: saving document #5000
2018-11-03 14:09:40,996 : INFO : PROGRESS: saving document #6000
2018-11-03 14:09:41,053 : INFO : PROGRESS: saving document #7000
2018-11-03 14:09:41,118 : INFO : PROGRESS: saving document #8000
2018-11-03 14:09:41,172 : INFO : PROGRESS: saving document #9000
2018-11-03 14:09:41,224 : INFO : PROGRESS: saving document #10000
2018-11-03 14:09:41,269 : INFO : PROGRESS: saving document #11000
2018-11

In the previous cells, we created a corpus of documents represented as a stream of vectors. To continue, let’s fire up gensim and use that corpus:

In [15]:
from gensim import corpora, models, similarities

### Creating a transformation


The transformations are standard Python objects, typically initialized by means of a training corpus:

Different transformations may require different initialization parameters; in case of TfIdf, the “training” consists simply of
going through the supplied corpus once and computing document frequencies of all its features.
Training other models, such as Latent Semantic Analysis or Latent Dirichlet Allocation, is much more involved and,
consequently, takes much more time.

In [16]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

2018-11-03 14:09:51,592 : INFO : collecting document frequencies
2018-11-03 14:09:51,597 : INFO : PROGRESS: processing document #0
2018-11-03 14:09:51,675 : INFO : PROGRESS: processing document #10000
2018-11-03 14:09:51,707 : INFO : calculating IDF weights for 15093 documents and 27866 features (260728 matrix non-zeros)


### Note
Transformations always convert between two specific vector spaces. The same vector space (= the same set of feature ids) must be used for training as well as for subsequent vector transformations. Failure to use the same input feature space, such as applying a different string preprocessing, using different feature ids, or using bag-of-words input vectors where TfIdf vectors are expected, will result in feature mismatch during transformation calls and consequently in either garbage output and/or runtime exceptions.

From now on, tfidf is treated as a read-only object that can be used to apply a transformation to a whole corpus:

In [17]:
corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors

### LDA:
https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation

Latent Dirichlet Allocation, LDA is yet another transformation from bag-of-words counts into a topic space of lower dimensionality. LDA is a probabilistic extension of LSA (also called multinomial PCA), so LDA’s topics can be interpreted as probability distributions over words. These distributions are, just like with LSA, inferred automatically from a training corpus. Documents are in turn interpreted as a (soft) mixture of these topics (again, just like with LSA).

In [18]:
total_topics = 5

In [19]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2018-11-03 14:10:09,896 : INFO : using symmetric alpha at 0.2
2018-11-03 14:10:09,901 : INFO : using symmetric eta at 0.2
2018-11-03 14:10:09,909 : INFO : using serial LDA version on this node
2018-11-03 14:10:09,938 : INFO : running online (single-pass) LDA training, 5 topics, 1 passes over the supplied corpus of 15093 documents, updating model once every 2000 documents, evaluating perplexity every 15093 documents, iterating 50x with a convergence threshold of 0.001000
2018-11-03 14:10:09,941 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2018-11-03 14:10:09,945 : INFO : PROGRESS: pass 0, at document #2000/15093
2018-11-03 14:10:12,375 : INFO : merging changes from 2000 documents into a model of 15093 documents
2018-11-03 14:10:12,439 : INFO : topic #0 (0.200): 0.040*"في" + 0.030*"البحرين" + 0.027*"من" + 0.021*"بصوتك" + 0.020*"تقدر" + 0.019*"الانتخابات" + 0.019*"انتخابات" + 0.013*"علي" + 0.008*"النيا

2018-11-03 14:10:20,382 : INFO : PROGRESS: pass 0, at document #14000/15093
2018-11-03 14:10:21,579 : INFO : merging changes from 2000 documents into a model of 15093 documents
2018-11-03 14:10:21,610 : INFO : topic #0 (0.200): 0.046*"في" + 0.033*"البحرين" + 0.021*"من" + 0.012*"ملكه" + 0.012*"علي" + 0.011*"مع" + 0.007*"وكاله" + 0.007*"العربيه" + 0.006*"السعوديه" + 0.005*"التعاون"
2018-11-03 14:10:21,613 : INFO : topic #1 (0.200): 0.046*"اخبار" + 0.030*"البحرين" + 0.028*"•" + 0.018*"في" + 0.015*"انتخابات" + 0.012*"٢٠١٨" + 0.011*"خارجيه" + 0.010*"من" + 0.009*"علي" + 0.008*"مجلس"
2018-11-03 14:10:21,617 : INFO : topic #2 (0.200): 0.027*"في" + 0.021*"وزير" + 0.016*"علي" + 0.014*"البحرين" + 0.014*"من" + 0.014*"الخارجيه" + 0.011*"معالي" + 0.011*"انباء" + 0.008*"وكاله" + 0.008*"الداخليه"
2018-11-03 14:10:21,621 : INFO : topic #3 (0.200): 0.017*"من" + 0.017*"في" + 0.014*"بن" + 0.011*"عن" + 0.010*"الوزراء" + 0.010*"شارع" + 0.010*"علي" + 0.008*"البحرين" + 0.008*"الملك" + 0.008*"خليفه"
2018-11-03

In [20]:
#Show first n important word in the topics:
lda.show_topics(total_topics,6)

[(0,
  '0.046*"في" + 0.030*"البحرين" + 0.019*"من" + 0.011*"علي" + 0.011*"مع" + 0.010*"ملكه"'),
 (1,
  '0.065*"اخبار" + 0.024*"البحرين" + 0.019*"في" + 0.019*"•" + 0.011*"انتخابات" + 0.010*"من"'),
 (2,
  '0.027*"في" + 0.019*"وزير" + 0.015*"علي" + 0.013*"من" + 0.013*"البحرين" + 0.011*"الخارجيه"'),
 (3,
  '0.017*"في" + 0.016*"بن" + 0.016*"من" + 0.011*"الوزراء" + 0.010*"عن" + 0.009*"علي"'),
 (4,
  '0.078*"الايام" + 0.043*"اخبار" + 0.018*"من" + 0.015*"في" + 0.013*"علي" + 0.010*"مع"')]

In [21]:
from collections import OrderedDict

data_lda = {i: OrderedDict(lda.show_topic(i,25)) for i in range(total_topics)}
#data_lda

In [22]:
import pandas as pd

df_lda = pd.DataFrame(data_lda)
print(df_lda.shape)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)

(85, 5)
(5, 85)


In [23]:
df_lda

,اخبار,الاخبار,الام,الامن,الانتخابات,الانسان,الايام,البحرين,الترشح,التعاون,...,واتساب,وزاره,وزير,وكاله,ولي,يتلقي,يستقبل,يشارك,٢٠١٨,•
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.003195,0.004706,0.029513,0.000000,0.005877,...,0.000000,0.00000,0.002896,0.005703,0.000000,0.003982,0.000000,0.002705,0.00000,0.000000
1,0.065400,0.004327,0.000000,0.000000,0.004954,0.000000,0.004027,0.024189,0.004955,0.000000,...,0.003057,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00805,0.018795
2,0.000000,0.000000,0.000000,0.005304,0.000000,0.000000,0.000000,0.013216,0.000000,0.000000,...,0.000000,0.00616,0.018989,0.006224,0.000000,0.000000,0.005532,0.000000,0.00000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005707,0.007205,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
4,0.042516,0.000000,0.002841,0.000000,0.000000,0.000000,0.078321,0.007026,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.004972,0.000000,0.000000,0.000000,0.00000,0.000000


In [25]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
2      30.037889        1       1 -125.617325  129.341049
0      29.440340        1       2   86.404785  115.015381
1      18.984097        1       3 -114.326126  -75.769325
3      10.880450        1       4  162.720520  -75.056007
4      10.657227        1       5   19.242575 -231.820358, topic_info=     Category        Freq      Term       Total  loglift  logprob
term                                                             
6025  Default  264.000000     اخبار  264.000000  30.0000  30.0000
3384  Default  167.000000    الايام  167.000000  29.0000  29.0000
5383  Default   60.000000         •   60.000000  28.0000  28.0000
1093  Default  102.000000      وزير  102.000000  27.0000  27.0000
1139  Default   56.000000        بن   56.000000  26.0000  26.0000
1678  Default   28.000000   الوزراء   28.000000  25.0000  25.0000
10    Default   37.000000  انتخابات   37.000000  24.0000  24.0000
3129  Default   57.000000  الخارجيه   57.000000  23.0000  23.0000
4119  Default   24.000000    خارجيه   24.000000  22.0000  22.0000
1637  Default   41.000000     الملك   41.000000  21.0000  21.0000
120   Default  228.000000        من  228.000000  20.0000  20.0000
644   Default   27.000000      ٢٠١٨   27.000000  19.0000  19.0000
57    Default   86.000000        مع   86.000000  18.0000  18.0000
4836  Default   48.000000     انباء   48.000000  17.0000  17.0000
5619  Default   13.000000      شارع   13.000000  16.0000  16.0000
6634  Default   23.000000    الدفاع   23.000000  15.0000  15.0000
4837  Default   53.000000     وكاله   53.000000  14.0000  14.0000
2516  Default   27.000000   التعاون   27.000000  13.0000  13.0000
1243  Default   47.000000     معالي   47.000000  12.0000  12.0000
114   Default   60.000000        عن   60.000000  11.0000  11.0000
77    Default   22.000000    النواب   22.000000  10.0000  10.0000
2796  Default   30.000000   العربيه   30.000000   9.0000   9.0000
1143  Default   16.000000     خليفه   16.000000   8.0000   8.0000
1149  Default   54.000000      ملكه   54.000000   7.0000   7.0000
2183  Default   25.000000     الامن   25.000000   6.0000   6.0000
272   Default   67.000000      رءيس   67.000000   5.0000   5.0000
3445  Default   19.000000      حادث   19.000000   4.0000   4.0000
2158  Default   33.000000  الداخليه   33.000000   3.0000   3.0000
84    Default   44.000000      مجلس   44.000000   2.0000   2.0000
1     Default  283.000000   البحرين  283.000000   1.0000   1.0000
...       ...         ...       ...         ...      ...      ...
5442   Topic5    0.886920    وصناعه    1.001918   2.1170  -7.5031
1047   Topic5    1.490710    ارتفاع    1.861274   2.0169  -6.9838
1098   Topic5    3.938926   التواصل    6.219802   1.7821  -6.0121
7877   Topic5    1.440485      توقع    1.819612   2.0053  -7.0181
6025   Topic5   68.386810     اخبار  264.521790   0.8862  -3.1579
2106   Topic5    1.886716      خدمه    2.635264   1.9048  -6.7482
2856   Topic5    7.997108       ولي   18.508114   1.3998  -5.3040
955    Topic5    2.369804     اعمال    3.603482   1.8198  -6.5202
1549   Topic5    7.337482     العهد   18.482836   1.3151  -5.3901
410    Topic5    6.045887        ما   16.065224   1.2617  -5.5837
5564   Topic5    3.935367         ل    8.530799   1.4653  -6.0130
57     Topic5   16.707537        مع   86.388504   0.5959  -4.5672
120    Topic5   28.778419        من  228.832916   0.1656  -4.0234
467    Topic5    3.659401       هذه    8.682704   1.3749  -6.0858
34     Topic5   20.230320       علي  178.836945   0.0596  -4.3759
23     Topic5   23.894533        في  434.461060  -0.6615  -4.2094
1135   Topic5    3.965426       اله   13.364382   1.0240  -6.0054
296    Topic5    3.981936       لدي   14.492371   0.9471  -6.0013
1145   Topic5    5.060847       سمو   29.981773   0.4599  -5.7615
1      Topic5   11.300495   البحرين  283.496674  -0.9834  -4.9582
220    Topic5    4.793819        ان   30.137407   0.4

In [28]:
lda.print_topics(num_topics=7)

2018-11-03 14:41:07,051 : INFO : topic #0 (0.200): 0.046*"في" + 0.030*"البحرين" + 0.019*"من" + 0.011*"علي" + 0.011*"مع" + 0.010*"ملكه" + 0.006*"العربيه" + 0.006*"السعوديه" + 0.006*"التعاون" + 0.006*"وكاله"
2018-11-03 14:41:07,054 : INFO : topic #1 (0.200): 0.065*"اخبار" + 0.024*"البحرين" + 0.019*"في" + 0.019*"•" + 0.011*"انتخابات" + 0.010*"من" + 0.009*"علي" + 0.008*"مجلس" + 0.008*"خارجيه" + 0.008*"٢٠١٨"
2018-11-03 14:41:07,057 : INFO : topic #2 (0.200): 0.027*"في" + 0.019*"وزير" + 0.015*"علي" + 0.013*"من" + 0.013*"البحرين" + 0.011*"الخارجيه" + 0.008*"معالي" + 0.008*"رءيس" + 0.008*"انباء" + 0.007*"الداخليه"
2018-11-03 14:41:07,060 : INFO : topic #3 (0.200): 0.017*"في" + 0.016*"بن" + 0.016*"من" + 0.011*"الوزراء" + 0.010*"عن" + 0.009*"علي" + 0.009*"الملك" + 0.008*"شارع" + 0.008*"رءيس" + 0.007*"خليفه"
2018-11-03 14:41:07,062 : INFO : topic #4 (0.200): 0.078*"الايام" + 0.043*"اخبار" + 0.018*"من" + 0.015*"في" + 0.013*"علي" + 0.010*"مع" + 0.007*"البحرين" + 0.005*"الدولي" + 0.005*"ولي" + 0.005

[(0,
  '0.046*"في" + 0.030*"البحرين" + 0.019*"من" + 0.011*"علي" + 0.011*"مع" + 0.010*"ملكه" + 0.006*"العربيه" + 0.006*"السعوديه" + 0.006*"التعاون" + 0.006*"وكاله"'),
 (1,
  '0.065*"اخبار" + 0.024*"البحرين" + 0.019*"في" + 0.019*"•" + 0.011*"انتخابات" + 0.010*"من" + 0.009*"علي" + 0.008*"مجلس" + 0.008*"خارجيه" + 0.008*"٢٠١٨"'),
 (2,
  '0.027*"في" + 0.019*"وزير" + 0.015*"علي" + 0.013*"من" + 0.013*"البحرين" + 0.011*"الخارجيه" + 0.008*"معالي" + 0.008*"رءيس" + 0.008*"انباء" + 0.007*"الداخليه"'),
 (3,
  '0.017*"في" + 0.016*"بن" + 0.016*"من" + 0.011*"الوزراء" + 0.010*"عن" + 0.009*"علي" + 0.009*"الملك" + 0.008*"شارع" + 0.008*"رءيس" + 0.007*"خليفه"'),
 (4,
  '0.078*"الايام" + 0.043*"اخبار" + 0.018*"من" + 0.015*"في" + 0.013*"علي" + 0.010*"مع" + 0.007*"البحرين" + 0.005*"الدولي" + 0.005*"ولي" + 0.005*"العهد"')]

In [ ]:
#datafile = 'Bahrain_tweets_cleaned.csv'
datafile = 'Bahrain_tweets_cleaned_Arabic.csv'